In [11]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pickle


In [3]:
### reading the dataset

data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
### removing not required data

data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
### encoding the gender

label_gender_encode=LabelEncoder()
data['Gender']=label_gender_encode.fit_transform(data['Gender'])


In [8]:
### encoding the geography using one hot encoder

one_hot_geo=OneHotEncoder()
geo_encoded=one_hot_geo.fit_transform(data[['Geography']])

geo_encoded_df=pd.DataFrame(geo_encoded.toarray(),columns=one_hot_geo.get_feature_names_out(['Geography']))


data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [9]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
#### input and output or dependent and independent feature

x=data.drop('Exited',axis=1)
y=data['Exited']

In [12]:
#### training the data

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


###scaler

scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.fit_transform(x_test)

In [13]:
#### creating the pickle file


with open('label_gender_encode.pkl','wb') as file:
    pickle.dump(label_gender_encode,file)

with open('one_hot_geo.pkl','wb') as file:
    pickle.dump(one_hot_geo,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [14]:
### buliding the ANN model 


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
import pickle
from datetime import datetime

In [15]:
### using function to create a ANN model


model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)), ## hl1
    Dense(32,activation='relu'), ##hl2
    Dense(1,activation='sigmoid') ## o/p

]
)

model.summary()

c:\Users\user\Desktop\ANN\env\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
#### optimizer loss and metrics

model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [18]:
#### saving the logs file


logs= "logs/fit" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=logs,histogram_freq=1)

In [19]:


earlystopping_callback=EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)

In [20]:
### training the model

model.fit(
    x_train,y_train,validation_data=(x_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,earlystopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7944 - loss: 0.4545 - val_accuracy: 0.8285 - val_loss: 0.3923
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8435 - loss: 0.3836 - val_accuracy: 0.8520 - val_loss: 0.3585
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8568 - loss: 0.3541 - val_accuracy: 0.8580 - val_loss: 0.3469
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8606 - loss: 0.3433 - val_accuracy: 0.8525 - val_loss: 0.3509
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8614 - loss: 0.3386 - val_accuracy: 0.8545 - val_loss: 0.3486
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8635 - loss: 0.3361 - val_accuracy: 0.8560 - val_loss: 0.3471
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8625 - loss: 0.3332 - val_accuracy: 0.8545 - val_loss: 0.3516
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8658 - loss: 0.3307 - val_accu

In [21]:
#### save the model file

model.save("model.h5")

In [25]:
### load extension board

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [28]:
### launching the tensorboard

%tensorboard --logdir logs/fit20260221-213506

Reusing TensorBoard on port 6006 (pid 9252), started 0:00:04 ago. (Use '!kill 9252' to kill it.)